In [1]:
import pathlib
import json
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
import numpy as np

import scipy.stats


In [100]:
Record = namedtuple('Record', 
                    ['header', 'train_loss', 'test_loss'])

def parse_file(fname):
    train_loss = []
    test_loss = []
    
    with open(fname, 'r') as f:
        for _ in range(3): f.readline()
        
        header = eval(f.readline())
        
        header = [x.split('=') for x in header if '=' in x]
        header = [[x[0][2:], x[1]] for x in header]
        header = dict(header)
        
        f.readline()

        for line in f:
            if line.startswith('#') or line.startswith('submitit') or 'tensor' in line: continue
            
            try:
                line = json.loads(line)
            except:
                print(line)
                return
            
            if 'mode' in line and line['mode'] == 'train':
                train_loss.append(line['acc'])
                continue
            if 'mode' in line and line['mode'] == 'test':
                test_loss.append(line['acc'])
                continue        

    return Record(header=header, train_loss=train_loss, test_loss=test_loss)                
            

In [101]:
def time_to_stop(log):
    import math
    for i, score in enumerate(log.train_loss):
        if score == 1.0:
            break
    assert not math.isnan(i)
    return i

In [102]:
def get_language_name(datum):
    name = datum.header['language']
    #position_scrambled = False #datum.header['scramble_positions'] == '1'
    #loss = datum.header['loss_type']
    #mixer = str(datum.header['mixers'])
    return name #+ '-' + mixer #('-pos_scrambled' if position_scrambled else '')

In [103]:
paths = [
    #nv =31
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_05_44_19/',
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_06_29_17/'
    # nv 11
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_06_57_04/'
    # nv 17
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_03_46'
    # nv 17 shuffled data
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_12_13/'
    
    # nv 31 shuffled
    # lstm
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_26_14/',
    # gru
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_08_31_11/',
    # gru add
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_08_50_29/'
    
    # redo everything with test (lstm, gru, gru add)
    '/private/home/kharitonov/nest/compositional_efficiency/2020_02_09_03_33_26/'
]

data = []

for path in paths:
    for fname in pathlib.Path(path).rglob('*.out'):
        parsed = parse_file(fname)
        data.append(parsed)

In [104]:
data[0].train_loss

[0.11464285850524902,
 0.7167857170104981,
 0.9425,
 0.97375,
 0.995,
 0.9880357170104981,
 0.99625,
 1.0]

In [116]:
language2caption = {
    'identity': 'Identity',
    'rotated': 'Rotated',
}


for task in ["autoenc", "linear", "mixed"]:
    for receiver_cell in ['lstm']:
                filtered = [d for d in data if \
                            d.header['loss_type'] == task and \
                            d.header['receiver_cell'] == receiver_cell and \
                            d.train_loss[-1] == 1.0
                           ]
                
                assert len(filtered) > 0
                
                languages = sorted(list(set(get_language_name(x) for x in filtered)))
                
                print("Task:", task)
                
                for language in languages:
                    measurements = [d for d in filtered if get_language_name(d) == language]
                    n = len(measurements)
                    time = [time_to_stop(x) for x in measurements]

                    mean, std = np.mean(time), np.std(time) / np.sqrt(n)
                    print(f"\t{language}, mean: {mean}, std: {std}")


Task: autoenc
	identity, mean: 6.2, std: 0.36878177829171543
	rotated, mean: 24.2, std: 3.088041450499005
Task: linear
	identity, mean: 25.4, std: 0.7099295739719539
	rotated, mean: 28.6, std: 2.1456933611306157
Task: mixed
	identity, mean: 23.4, std: 1.9959959919799437
	rotated, mean: 5.4, std: 0.25298221281347033


In [118]:
language2caption = {
    'identity': 'Identity',
    'rotated': 'Rotated',
}


for task in ["autoenc", "linear", "mixed"]:
    for receiver_cell in ['gru']:
                filtered = [d for d in data if \
                            d.header['loss_type'] == task and \
                            d.header['receiver_cell'] == receiver_cell and \
                            d.train_loss[-1] == 1.0
                           ]
                
                assert len(filtered) > 0
                
                languages = sorted(list(set(get_language_name(x) for x in filtered)))
                
                print("Task:", task)
                
                for language in languages:
                    measurements = [d for d in filtered if get_language_name(d) == language]
                    n = len(measurements)
                    acc = [x.test_loss[-1] for x in measurements]

                    mean, std = np.mean(acc), np.std(acc) / np.sqrt(n)
                    print(f"\t{language}, mean: {mean}, std: {std}")


Task: autoenc
	identity, mean: 0.9466319626285914, std: 0.011364667742259787
	rotated, mean: 0.09244882209427224, std: 0.017814546368060246
Task: linear
	identity, mean: 0.0002232142857142857, std: 0.00021756237376805722
	rotated, mean: 0.0, std: 0.0
Task: mixed
	identity, mean: 0.06329070096392007, std: 0.011520032923165285
	rotated, mean: 0.9675754708903176, std: 0.00335899570636066


In [39]:
def mean_std(data):
    means = []
    stds = []
    
    losses = [d.test_loss for d in data]
    
    for l in zip(*losses):
        means.append(np.mean(l))
        stds.append(np.std(l) / np.sqrt(len(l)))
        
    return means, stds

In [40]:
for slice, name, color in [
        (no_rotation, 'no rotation', 'r'), 
        (rotation, 'rotation', 'black'),
        (swap_data, 'swap', 'blue')]:
    means, stds = mean_std(slice)
    xs = list(range(1, len(means) + 1))
    #plt.plot(means[:3], label=name, color=color)
    plt.errorbar(x=xs[:20], y=means[:20], yerr=stds[:20], color=color, alpha=0.75, label=name)
plt.xlabel('Epoch')
plt.ylabel('MSE')

plt.legend()

NameError: name 'no_rotation' is not defined

In [171]:
mean_std(rotation)

NameError: name 'rotation' is not defined